In [ ]:
#Axtell (1999), the emergence of firms
#version 0.8

import random
from math import sqrt
import matplotlib.pyplot as plt
import time

N=50 #número de agentes
v=2 #múmero de vecinos, no está seteado el codigo para cambiar esto
agentList=[]
firmList=[]
a,b,beta=1,1,2

In [ ]:
class agent:
    
    global N, agentList, v, a, b ,beta
    def __init__(self,preferences,v):
       
        self.preferences=preferences #los agentes se inician preferencias
        agentList.append(self) #arma la lista de agentes
        self.effort=0 #se inician con efuerzo 0
        self.lastIncome=0 #con ingreso 0
        self.lastUtility=0 #y utilidad 0
        self.currentFirm=[] #se inicializan sin firma

    def defineNeighbors(self,agentList,v):
        self.neighbors=random.sample([others for others in agentList if others!=self],v)
        
    #computar el esfuerzo dentro de la firma actual
    def computeOptimalEffortInFirm(self):
        totalEffort=self.currentFirm.totalEffort
        othersEffort=totalEffort-self.effort
        optimalEffortInFirm=max(0,(-a-2*b*(othersEffort-self.preferences)+sqrt(
            a**2+4*a*b*self.preferences**2*(1+othersEffort)+(
            2*b*self.preferences*(1+othersEffort))**2))/(
            2*b*(1+self.preferences)))
        expOutputInFirm=a*(othersEffort+optimalEffortInFirm)+b*(othersEffort+optimalEffortInFirm)**beta
        expIncomeInFirm=expOutputInFirm/self.currentFirm.size
        expUtilityInFirm=(expIncomeInFirm**self.preferences
                                 *(1-optimalEffortInFirm)**(1-self.preferences))
        'chequear la salida'
        return {'e':optimalEffortInFirm,'o':expOutputInFirm,'i':expIncomeInFirm,'u':expUtilityInFirm}
    
    def computeOptimalEffortInNewFirm(self):
        othersEffort=0
        optimalEffortInNewFirm=max(0,(-a-2*b*(othersEffort-self.preferences)+sqrt(
            a**2+4*a*b*self.preferences**2*(1+othersEffort)+(
            2*b*self.preferences*(1+othersEffort))**2))/(
            2*b*(1+self.preferences)))
        
        expOutputInNewFirm=a*(optimalEffortInNewFirm)+b*(optimalEffortInNewFirm)**beta
        expIncomeInNewFirm=expOutputInNewFirm
        expUtilityInNewFirm=(expIncomeInNewFirm**self.preferences
                                 *(1-optimalEffortInNewFirm)**(1-self.preferences))
        'chequear la salida'
        return {'e':optimalEffortInNewFirm,'o':expOutputInNewFirm,'i':expIncomeInNewFirm,'u':expUtilityInNewFirm}
       
       
    
    def computeOptimalEffortInNearFirms(self):
        optimalEffortInNearFirms={}
        expOutputInNearFirms={}
        expIncomeInNearFirms={}
        expUtilityInNearFirms={}
        
        for i in self.neighbors:
            othersEffort=i.currentFirm.totalEffort
            optimalEffort=max(0,(-a-2*b*(othersEffort-self.preferences)+sqrt(
            a**2+4*a*b*self.preferences**2*(1+othersEffort)+(
            2*b*self.preferences*(1+othersEffort))**2))/(
            2*b*(1+self.preferences)))
            
            expOutput=a*(optimalEffort+othersEffort)+b*(optimalEffort+othersEffort)**beta
            expIncome=expOutput/i.currentFirm.size
            expUtility=(expIncome**self.preferences
                                     *(1-optimalEffort)**(1-self.preferences))

            optimalEffortInNearFirms.update({i.currentFirm:optimalEffort})
            expOutputInNearFirms.update({i.currentFirm:expOutput})
            expIncomeInNearFirms.update({i.currentFirm:expIncome})
            expUtilityInNearFirms.update({i.currentFirm:expUtility})
        'chequear la salida de esta funcion'
        return {'e':optimalEffortInNearFirms,'o':expOutputInNearFirms,'i':expIncomeInNearFirms,'u':expUtilityInNearFirms}
     
        
     
        
    #computar la utilidad
    def computeUtility(self):
        self.lastUtility=self.lastIncome**self.preferences*(1-self.effort)**(1-self.preferences)
        

    #cambiar de firma
    def changeFirm(self,expUtilityInCurrentFirm,expUtilityInNewFirm,expUtilityInNearFirms,effortInCurrentFirm,effortInNewFirm,effortInNearFirms):
        utilityOptions=[expUtilityInCurrentFirm,expUtilityInNewFirm]
        for v in expUtilityInNearFirms.values():
            utilityOptions.append(v)
        print(utilityOptions)    
        maxUtility=max(utilityOptions)
        chosenOption=utilityOptions.index(maxUtility)
        if chosenOption==1: #elige fundar firma
            for i in firmList:
                if i.size==0:
                    newFirm=i
                    self.effort=effortInNewFirm
                    break
                self.currentFirm=newFirm
                self.effort=effortInNewFirm
                return newFirm
        elif  chosenOption==0: #quedarse en la firma
            self.effort=effortInCurrentFirm
            return 'stay'
        
        else:  #elige un vecino
            newFirm=self.neighbors[chosenOption-2].currentFirm
            self.currentFirm=newFirm
            print(effortInNearFirms)
            print(self.neighbors[chosenOption-2])
            self.effort=effortInNearFirms[self.neighbors[chosenOption-2].currentFirm]
            return newFirm
        


class firm:
    global N, agentList, v, a, b, beta
    def __init__(self,founder):
        self.founder=founder #para inicializar una firma con un agente
        firmList.append(self) #construye la lista de firmas
        self.employees=[founder]
        
        self.lastOutput=0
        self.totalEffort=0
    
    def computeSize(self):
        self.size=len(self.employees)
        return self.size
    
    
    #computa el esfuerzo total en la firma
    def computeTotalEffort(self):
        totalEffort=0
        for i in self.employees:
            print(i,' ',i.effort)
            totalEffort+=i.effort
        print(totalEffort)
        self.totalEffort=totalEffort
        
    
    #computa el producto    
    def computeOutput(self):
        self.lastOutput=a*(self.totalEffort)+b*(self.totalEffort)**beta

        
    #paga salarios
    def payIncome(self):
        try:
            wage=self.lastOutput/self.size
            for i in self.employees:
                i.lastIncome=wage
        except ZeroDivisionError:
            wage=0
            
        
        
    
    
    #quita un agente de la firma
    def removeAgent(self,agentRemoved):
        self.employees.remove(agentRemoved)
    
    #agrega un agente en la fima
    def addAgent(self,agentAdded):
        self.employees.append(agentAdded)       
        
        
    
    
        
        
        
        
        

In [ ]:
def test(i): #test para ver firmas y agentes
    global agentList,firmList
    print('*****agent'+str(i)+'****')
    print(agentList[i])
    print('thita: ',agentList[i].preferences)
    print('firma actual: ',agentList[i].currentFirm)
    print('esfuerzo: ',agentList[i].effort)
    print('ingreso: ', agentList[i].lastIncome)
    print('utilidad: ', agentList[i].lastUtility)
    try:
        print('vecinos',agentList[i].neighbors)
    except: 
        pass
    print()
    print('*****firm'+str(i)+'****')
    print(firmList[i])
    print('nombre de empleados: ',firmList[i].employees)
    try:
        print('tamaño',firmList[i].size)
    except:
        pass
    print('esfuerzo total: ',firmList[i].totalEffort)
    print('producto: ',firmList[i].lastOutput)
    print()

In [ ]:
#condiciones iniciales



agentList=[] #borra todo agente previo
firmLine=[] #borra toda firma previa

#crea N firmas y agentes

for i in range(N):
    titha=random.random()
    agent(titha,v).position=i
    

    
for each in agentList:
    each.defineNeighbors(agentList,v) #toma vecinos al azar
    each.currentFirm=firm(each) #cada agente funda incialmente una firma
    returnedDictionary=each.computeOptimalEffortInNewFirm() #computa esfuerzo e ingreso incial
    each.effort=returnedDictionary['e']
    each.lastIncome=returnedDictionary['i']
    each.lastUtility=returnedDictionary['u']
n=0
for each in firmList: #computa el esfuerzo total en la firma y el producto para el estado incial
    each.computeTotalEffort()
    each.computeOutput()
    each.position=n
    each.computeSize()
    n+=1

In [ ]:
for i in range(N):
    test(i)

In [ ]:
# comienza la simulación
def simulation(steps,iterations):
    global agentList, firmList, N
    plt.ion()
    plt.figure()
    plt.axis([-1, N+1, -1, N])
    
    
    for h in range(iterations):
        print('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFf')
        agentsActivated=random.sample(agentList,steps)
        for i in agentsActivated:
            
            print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
            
            oldFirm=i.currentFirm
            returnedDictInSameFirm=i.computeOptimalEffortInFirm()
            returnedDictInNewFirm=i.computeOptimalEffortInNewFirm()
            returnedDictInNearFirms=i.computeOptimalEffortInNearFirms()
            print('utility in same firm ',returnedDictInSameFirm['u'],'\n utility in new firm',
                  returnedDictInNewFirm['u'], '\n ut in near firms ', returnedDictInNearFirms['u'])

            newFirm=i.changeFirm(returnedDictInSameFirm['u'],returnedDictInNewFirm['u'],
                                returnedDictInNearFirms['u'],returnedDictInSameFirm['e'],
                                returnedDictInNewFirm['e'],
                                returnedDictInNearFirms['e'])
            if newFirm=='stay':
                break
            else: 
                newFirm.addAgent(i)
                oldFirm.removeAgent(i)
            
            
            xdata=range(N)#graficar el tamaño contra la firma
            ydata=[]
            for l in firmList:
                a=l.computeSize()
                ydata.append(a)
            
            plt.cla() 
            g0=plt.plot(xdata,ydata,'ro')
            plt.setp(g0,ms=15)
            plt.draw()
            plt.pause(0.05)
            
            

        for j in firmList:
            j.computeTotalEffort()
            j.computeOutput()
            j.payIncome()

        for k in agentList:
            k.computeUtility()
        
        for p in range(N):
            test(p)
        
        
      


In [ ]:
simulation(10,50)